In [7]:
# !pip3 install line_profiler
import pandas as pd
import os
import pyodbc
import time
from line_profiler import LineProfiler
from multiprocessing import Pool

In [10]:
conn = pyodbc.connect("DSN=CarmelODBCDSII", autocommit = True)

sql_query='select user_id as seller_id from P_AML_KEL_TEMP_T.seller2list'
seller_id = pd.read_sql(sql_query, con=conn)

In [9]:
hash_table=pd.read_csv('data/hash_table.csv',index_col=0)
hash_table.buyer_id=hash_table.buyer_id.astype('int')
print(hash_table.shape)

# sql_query=\
# 'select xid.user_id as buyer_id, xid.link_strtgy_match_hash \
# from incdata_v.USER_LINK_STRTGY_MATCH as xid \
# where xid.link_strtgy_match_hash in (select distinct link_strtgy_match_hash from P_AML_KEL_TEMP_T.matchedhash)'

# hash_table = pd.read_sql(sql_query, con=conn)
# hash_table.to_csv('data/hash_table.csv')

/usr/local/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(4750249, 2)


In [11]:
seller_id_list=list(seller_id.seller_id.astype('int'))

In [44]:
hash_table['hash'] = hash_table['link_strtgy_match_hash'].apply(hash)

In [49]:

start=time.time()
# hash_table[['hash','link_strtgy_match_hash']].head()
hash_table['hash'] = hash_table['link_strtgy_match_hash'].apply(hash)
end=time.time()
print(end-start)

1.097640037536621


In [50]:
%load_ext line_profiler
def process_func(k):
    temp=hash_table.loc[hash_table.hash==k]
    output=[]
    for i in temp.buyer_id:
        if i in seller_id_list:
            output.append([(i,j) for j in temp.buyer_id if j != i])
    return [item for sublist in output for item in sublist]

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [43]:
%lprun -f process_func process_func(hash_table.hash.unique()[142])

In [87]:
for i in hash_table.hash.unique()[:10]:
    print(i)

509421922895602295
1543331710393340106
-6699733011763924496
-7742826108410402212
5661751196036368974
8608316101807604390
435673993467575665
4594382673533151757
7911140825057110936
1697113614088513325


In [92]:
output=[]
start=time.time()
# for i,j in enumerate(hash_table.hash.unique()):
#     if i%1000==0:
#         output.append(j)
output=[j for i,j in enumerate(hash_table.hash.unique()) if i%1000==0]
end=time.time()
print(end-start)

0.3300907611846924


In [84]:
1050%1000

50

In [72]:
smp_list=[j for i,j in enumerate(hash_table.hash.unique()) if i%1000==0]

In [75]:
2.5*1000/3600

0.6944444444444444

In [74]:
start=time.time()
pool = Pool(processes=os.cpu_count())
grand_output=pool.map(process_func, smp_list)
pool.close()
pool.join()
end=time.time()
print(end-start)

2.548079013824463


In [69]:
hash_table.hash.nunique()

1580698

In [70]:
gg=[item for sublist in grand_output for item in sublist]

In [71]:
len(list(set(gg)))

1674

In [ ]:
with open('data/grand_output.pkl', 'wb') as f:
    pickle.dump(grand_output, f)